In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import us
import copy
import tqdm.auto as tqdm
import geopandas as gpd
import numpy as np
import pandas as pd
from shapefiles import shapefiles, school_district_shapefiles, ce_to_name
from relationship import full_relationships, create_relationships
from stats_for_shapefile import compute_statistics_for_shapefile, compute_summed_shapefile_all_keys, sum_keys
from create_website import (
    full_shapefile,
    shapefile_without_ordinals,
    add_ordinals,
    get_statistic_categories,
    create_page_jsons,
)
from election_data import with_election_results
from urbanstats.features.extract_data import feature_data
import matplotlib.pyplot as plt
import matplotlib as mpl

from urbanstats.data.gpw import *
from urbanstats.data.population_urban_area import *

from collections import defaultdict, Counter
import requests

from shapefiles import CONGRESSIONAL_DISTRICTS, COUNTIES, county_cross_cd, usda_county_type, shapefiles

/home/kavi/anaconda3/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [3]:
import io


In [4]:
from urbanstats.osm.buses import national_stops
from urbanstats.osm.trains import *

In [5]:
urls = gtfs_list()

In [6]:
url = "feeds/lacmta.github.com.dmfr.json"

In [7]:
# static_url = static_urls[9]

In [8]:
for url in tqdm.tqdm(urls):
    for feed in read_gtfs_spec(url)["feeds"]:
        feed = feed["urls"]
        if all(x.startswith("realtime") for x in feed):
            continue
        static_url = feed["static_current"]
#         print(static_url)
        try:
            read_gtfs_from_url_raw(static_url)
        except Exception as e:
            print(e)

  0%|          | 0/654 [00:00<?, ?it/s]

No connection adapters were found for 'ftp://ftp.coruna.es/gtfs/busurbano-coruna-es.zip'


KeyError: 'static_current'

In [398]:
[x["urls"] for x in read_gtfs_spec(url)["feeds"]]

[{'static_current': 'http://api.511.org/transit/datafeeds?operator_id=SF',
  'static_historic': ['https://gtfs.sfmta.com/transitdata/google_transit.zip']},
 {'static_current': 'https://www.capitolcorridor.org/googletransit/GTFS.zip'},
 {'static_current': 'http://api.511.org/transit/datafeeds?operator_id=RG'},
 {'realtime_vehicle_positions': 'https://api.511.org/Transit/VehiclePositions?agency=RG',
  'realtime_trip_updates': 'https://api.511.org/Transit/TripUpdates?agency=RG',
  'realtime_alerts': 'https://api.511.org/Transit/servicealerts?agency=RG'}]

In [387]:
import pickle
sum(len(pickle.dumps(read_gtfs_from_url(static_url))) for static_url in static_urls) / 1e6

7.188972

{'agency.txt':    agency_id                                         agency_url agency_lang  \
 0       1721  https://www.cityofsouthgate.org/218/GATE-Get-A...          en   
 
                agency_name  agency_phone      agency_timezone  \
 0  Get Around Town Express  323-563-5754  America/Los_Angeles   
 
    agency_fare_url  tts_agency_name  
 0              NaN              NaN  ,
 'calendar_attributes.txt':   service_id          service_description
 0         Sa  Year Round  (Saturday only)
 1       wkdy        Year Round  (Weekday),
 'calendar_dates.txt':   service_id      date      holiday_name  exception_type
 0       wkdy  20231123  Thanksgiving Day               2
 1       wkdy  20230529      Memorial Day               2
 2       wkdy  20230704  Independence Day               2
 3       wkdy  20230904         Labor Day               2
 4       wkdy  20240101    New Year's Day               2
 5       wkdy  20241128  Thanksgiving Day               2
 6       wkdy  20240527   

In [359]:
sum(len(requests.get(static_url).content) for static_url in static_urls) / 1e6a

1.868267

In [355]:
pd.read_csv(zip_file.open("agency.txt"))

agency_id                                         agency_url agency_lang  \
0       1671  https://www.westcovina.org/home/showdocument?i...          en   

       agency_name  agency_phone      agency_timezone  agency_fare_url  \
0  Go West Shuttle           NaN  America/Los_Angeles              NaN   

   tts_agency_name  
0              NaN

In [245]:
table = pd.read_csv("named_region_shapefiles/gtfs_sources.csv")

In [247]:
for _, row in tqdm.tqdm(table.iterrows(), total=len(table)):
    try:
        x, ag = gtfs_stops(row["urls.direct_download"])
    except Exception as e:
        print(row["provider"], ": ", e)
    

  0%|          | 0/1985 [00:00<?, ?it/s]

http://smttracker.com/downloads/gtfs/cascobaylines-portland-me-usa.zip
Casco Bay Lines :  File is not a zip file
http://www.myridebarrie.ca/gtfs/Google_transit.zip
https://transitfeeds.com/p/athens-public-transit/298/latest/download
https://s3.amazonaws.com/datatools-511ny/public/St_Lawrence_County_Public_Transit.zip
https://cdn.buenosaires.gob.ar/datosabiertos/datasets/sbase/subte-gtfs/subte-gtfs-zip.zip
Subterráneos de Buenos Aires (SUBTE) :  "There is no item named 'routes.txt' in the archive"
https://dadosabertos.poa.br/dataset/1fe9c2c1-9fbe-48ea-841b-61e30597ecd6/resource/b3bce61f-78ee-49eb-be57-6236d82bd5e0/download/outputfiles.zip
Empresa Publica de Transportes e Circulação (EPTC) :  'DataFrame' object has no attribute 'location_type'
https://www.sptrans.com.br/umbraco/Surface/PerfilDesenvolvedor/BaixarGTFS?memberName=sptrans
São Paulo Transporte (SPTrans) :  'DataFrame' object has no attribute 'location_type'
https://ckan.pbh.gov.br/dataset/77764a7e-63fc-4111-ace3-fb7d3037953a/

/home/kavi/anaconda3/lib/python3.7/site-packages/permacache/cache.py:33: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  value = self.function(*args, **kwargs)


http://www.gonctd.com/google_transit.zip
https://www.octa.net/current/google_transit.zip
http://data.trilliumtransit.com/gtfs/lagunabeach-ca-us/lagunabeach-ca-us.zip
http://www.ycipta.org/gtfs/google_transit.zip
http://data.trilliumtransit.com/gtfs/paloverde_valley-ca-us/paloverde_valley-ca-us.zip
http://data.trilliumtransit.com/gtfs/humboldtcounty-ca-us/humboldtcounty-ca-us.zip
http://data.trilliumtransit.com/gtfs/delnorte-ca-us/delnorte-ca-us.zip
http://oregon-gtfs.com/gtfs_data/cooscounty-or-us/cooscounty-or-us.zip
https://oregon-gtfs.trilliumtransit.com/gtfs_data/currypublictransit-or-us/currypublictransit-brookings-or-us.zip
http://oregon-gtfs.com/gtfs_data/lincolncounty-or-us/lincolncounty-or-us.zip
http://oregon-gtfs.com/gtfs_data/rhodyexpress-or-us/rhodyexpress-or-us.zip
http://data.trilliumtransit.com/gtfs/easternsierra-ca-us/easternsierra-ca-us.zip
http://data.trilliumtransit.com/gtfs/smat-ca-us/smat-ca-us.zip
http://data.trilliumtransit.com/gtfs/airportvaletexpress-ca-us/air

http://data.trilliumtransit.com/gtfs/tehama-ca-us/tehama-ca-us.zip
http://data.trilliumtransit.com/gtfs/siskiyou-ca-us/siskiyou-ca-us.zip
http://data.trilliumtransit.com/gtfs/lassen-ca-us/lassen-ca-us.zip
http://data.trilliumtransit.com/gtfs/plumas-ca-us/plumas-ca-us.zip
http://data.trilliumtransit.com/gtfs/rtc-nv-us/rtc-nv-us.zip
http://oregon-gtfs.com/gtfs_data/josephinecounty-or-us/josephinecounty-or-us.zip
http://data.trilliumtransit.com/gtfs/rvtd-or-us/rvtd-or-us.zip
http://oregon-gtfs.com/gtfs_data/umpquatransit-or-us/umpquatransit-or-us.zip
http://oregon-gtfs.com/gtfs_data/craterlaketrolley-or-us/craterlaketrolley-or-us.zip
http://oregon-gtfs.com/gtfs_data/basin-or-us/basin-or-us.zip
http://oregon-gtfs.com/gtfs_data/benton-or-us/benton-or-us.zip
http://oregon-gtfs.com/gtfs_data/caravan-or-us/caravan-or-us.zip
https://www.cherriots.org/media/doc/gtfs.zip
http://oregon-gtfs.com/gtfs_data/citytocityshuttle-or-us/citytocityshuttle-or-us.zip
http://oregon-gtfs.com/gtfs_data/diamondex

...

MOOver! :  File is not a zip file
https://api.goswift.ly/real-time/dc-streetcar/gtfs-rt-vehicle-positions
DC Streetcar :  File is not a zip file
https://api.goswift.ly/real-time/dc-streetcar/gtfs-rt-alerts/
DC Streetcar :  File is not a zip file
https://api.goswift.ly/real-time/dc-streetcar/gtfs-rt-trip-updates
DC Streetcar :  File is not a zip file
https://api.goswift.ly/real-time/avalon/gtfs-rt-vehicle-positions
Avalon Transit :  File is not a zip file
https://api.goswift.ly/real-time/avalon/gtfs-rt-trip-updates
Avalon Transit :  File is not a zip file
https://api.goswift.ly/real-time/via/gtfs-rt-trip-updates
Via Mobility :  File is not a zip file
https://api.goswift.ly/real-time/via/gtfs-rt-alerts
Via Mobility :  File is not a zip file
https://api.goswift.ly/real-time/via/gtfs-rt-vehicle-positions
Via Mobility :  File is not a zip file
http://www.vre.org/gtfs-realtime/TripUpdateFeed
Virginia Railway Express :  File is not a zip file
http://www.vre.org/gtfs-realtime/VehiclePositionFe

Commerce Municipal Bus Lines :  File is not a zip file
https://api.entur.io/realtime/v1/gtfs-rt/vehicle-positions?datasource=GOA
Go-Ahead :  File is not a zip file
https://api.entur.io/realtime/v1/gtfs-rt/trip-updates?datasource=RUT
Oslo region (Ruter) :  File is not a zip file
https://api.entur.io/realtime/v1/gtfs-rt/alerts?datasource=RUT
Oslo region (Ruter) :  File is not a zip file
https://api.entur.io/realtime/v1/gtfs-rt/trip-updates?datasource=VYB
Vy Buss (SE) :  File is not a zip file
https://api.entur.io/realtime/v1/gtfs-rt/vehicle-positions?datasource=VYB
Vy Buss (SE) :  File is not a zip file
https://api.entur.io/realtime/v1/gtfs-rt/vehicle-positions?datasource=VYB
Vy Buss (SE) :  File is not a zip file
https://api.entur.io/realtime/v1/gtfs-rt/vehicle-positions?datasource=VYX
Vy Express :  File is not a zip file
https://api.entur.io/realtime/v1/gtfs-rt/alerts?datasource=VYX
Vy Express :  File is not a zip file
https://api.entur.io/realtime/v1/gtfs-rt/trip-updates?datasource=VY

Belleville Transit :  'DataFrame' object has no attribute 'location_type'
https://data.gtfs.be/sncb/gtfs/be-sncb-gtfs.zip
https://data.gtfs.be/dewaterbus/gtfs/be-dewaterbus-gtfs.zip
https://drive.google.com/u/0/uc?id=18zwweT64NI5O_goGVc_h-OGTTShfiacG&export=download
Wave Transit :  'DataFrame' object has no attribute 'location_type'
https://drive.google.com/u/0/uc?id=1w_7Ma9B9-aA-iBTWz3gxwGlORXPjppQ-&export=download
Wave Transit :  'DataFrame' object has no attribute 'location_type'
https://dados.fortaleza.ce.gov.br/dataset/51afc610-d48b-4fa8-8dea-9c65747148c7/resource/fc9109ec-48a9-4af3-a7c8-4909cc1e4ac0/download/exportacao.zip
http://datos.gob.cl/dataset/cef4c471-2837-412b-a78e-1d4c6a261bf9/resource/7887a7e7-9af6-4fc8-b19f-8ff4ea474d1c/download/temuco24nov16.zip
http://datos.gob.cl/dataset/22cb7071-34f2-46dc-8c2d-dd9b86e9c32f/resource/65b65c20-a4ab-4cca-ba55-d2ef5bbb9605/download/rancagua1feb16.zip
https://datos.gob.cl/dataset/22fa7b82-50ec-4c85-aa89-709a8e0dd268/resource/f4526a2f-dd

/home/kavi/anaconda3/lib/python3.7/site-packages/permacache/cache.py:33: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  value = self.function(*args, **kwargs)


https://data.discoverpassenger.com/operator/bts/dataset/current/download/gtfs
https://data.discoverpassenger.com/operator/ccts/dataset/current/download/gtfs
https://data.discoverpassenger.com/operator/carouselbuses/dataset/current/download/gtfs
https://data.discoverpassenger.com/operator/eyms/dataset/current/download/gtfs
https://data.discoverpassenger.com/operator/gocornwallbus/dataset/current/download/gtfs
https://data.discoverpassenger.com/operator/gonortheast/dataset/current/download/gtfs
https://data.discoverpassenger.com/operator/gonorthwest/dataset/current/download/gtfs
https://data.discoverpassenger.com/operator/greenline/dataset/current/download/gtfs
https://data.discoverpassenger.com/operator/hedinghamandchambers/dataset/current/download/gtfs
https://data.discoverpassenger.com/operator/intalink/dataset/current/download/gtfs
https://data.discoverpassenger.com/operator/konectbus/dataset/current/download/gtfs
https://data.discoverpassenger.com/operator/mcgills/dataset/current/do

stop_code                                    stop_name  stop_desc  \
stop_id                                                                      
10056        10056  HALAULANI LEEWARD COMMUNITY COLLEGE STATION        NaN   
10055        10055                 HALAWA ALOHA STADIUM STATION        NaN   
10054        10054                   KALAUAO PEARLRIDGE STATION        NaN   
10053        10053               WAIAWA PEARL HIGHLANDS STATION        NaN   
10047        10047               KUALAKA'I EAST KAPOLEI STATION        NaN   
10046        10046              KEONE'AE U.H. WEST OAHU STATION        NaN   
10045        10045                  HONOULIULI HO'OPILI STATION        NaN   
10044        10044                   HO'AE'AE WEST LOCH STATION        NaN   
10043        10043       POUHALA WAIPAHU TRANSIT CENTER STATION        NaN   

          stop_lat    stop_lon  zone_id  \
stop_id                                   
10056    21.394324 -157.985603      NaN   
10055    21.370999 -157.933887      NaN   
10054    21.383876 -157.947094      NaN   
10053    21.396690 -157.979855      NaN   
10047    21.345574 -158.050995      NaN   
10046    21.358532 -158.051188      NaN   
10045    21.366980 -158.045203      NaN   
10044    21.378141 -158.022731      NaN   
10043    21.384093 -158.003818      NaN   

                                          stop_url  location_type  \
stop_id                                                             
10056    http://hea.thebus.org/nextbus.asp?s=10056            0.0   
10055    http://hea.thebus.org/nextbus.asp?s=10055            0.0   
10054    http://hea.thebus.org/nextbus.asp?s=10054            0.0   
10053    http://hea.thebus.org/nextbus.asp?s=10053            0.0   
10047    http://hea.thebus.org/nextbus.asp?s=10047            0.0   
10046    http://hea.thebus.org/nextbus.asp?s=10046            0.0   
10045    http://hea.thebus.org/nextbus.asp?s=10045            0.0   
10044    http://hea.thebus.org/nextbus.asp?s=10044            0.0   
10043    http://hea.thebus.org/nextbus.asp?s=10043            0.0   

         parent_station  stop_serial_number types  agency_id  
stop_id                                                       
10056               NaN                 NaN   [1]  [Skyline]  
10055               NaN                 NaN   [1]  [Skyline]  
10054               NaN                 NaN   [1]  [Skyline]  
10053               NaN                 NaN   [1]  [Skyline]  
10047               NaN                 NaN   [1]  [Skyline]  
10046               NaN                 NaN   [1]  [Skyline]  
10045               NaN                 NaN   [1]  [Skyline]  
10044               NaN                 NaN   [1]  [Skyline]  
10043               NaN                 NaN   [1]  [Skyline]

In [241]:
x = gtfs_stops(row["urls.direct_download"])

In [235]:
x[x.types.apply(bool)]

Empty DataFrame
Columns: [stop_code, stop_name, stop_desc, stop_lat, stop_lon, zone_id, stop_url, location_type, parent_station, stop_timezone, wheelchair_boarding, types, agency_id]
Index: []

In [120]:
row = table[table["location.municipality"] == "Atlanta"].iloc[3]

In [134]:
row = table[table["name"] == "NYC Subway"].iloc[0]

In [215]:
row = table[table["location.municipality"] == "Los Angeles"].iloc[2]

In [225]:
gtfs_stops(row["urls.direct_download"])

https://gitlab.com/LACMTA/gtfs_rail/raw/master/gtfs_rail.zip
{'80425': [0], '80132': [0], '80106': [0], '80705': [0], '80101': [0], '80126': [0], '80125': [0], '80416': [0], '80109': [0], '80414': [0], '80407': [0], '80306': [0], '80703': [0], '80708': [0], '80424': [0], '80419': [0], '80208': [1], '80427': [0], '80137': [0], '80108': [0], '80127': [0], '80113': [0], '80418': [0], '80426': [0], '80421': [0], '80139': [0], '80138': [0], '80102': [0], '80304': [0], '80213': [1], '80112': [0], '80153': [0], '80420': [0], '80204': [1], '80207': [1], '80423': [0], '81401': [0], '80128': [0], '80114': [0], '80121': [0], '80422': [0], '80403': [0], '80704': [0], '80211': [1], '80212': [1], '80707': [0], '81403': [0], '80215': [1], '80154': [0], '80201': [1], '80209': [1], '80123': [0], '80122': [0], '80404': [0], '81402': [0], '80309': [0], '80206': [1], '80124': [0], '80313': [0], '80136': [0], '80134': [0], '80133': [0], '80205': [1], '80307': [0], '80302': [0], '80115': [0], '80401': [0], 

stop_code                             stop_name  stop_desc   stop_lat  \
stop_id                                                                         
80101S     80101S           Downtown Long Beach Station        NaN  33.768071   
80102S     80102S                   Pacific Ave Station        NaN  33.772258   
80105S     80105S                Anaheim Street Station        NaN  33.781830   
80106S     80106S             Pacific Coast Hwy Station        NaN  33.789090   
80107S     80107S                 Willow Street Station        NaN  33.807079   
...           ...                                   ...        ...        ...   
80708S     80708S         Martin Luther King Jr Station        NaN  34.009563   
80709S     80709S        Expo / Crenshaw K-Line Station        NaN  34.022156   
81401S     81401S  Grand Ave Arts / Bunker Hill Station        NaN  34.054724   
81402S     81402S             Historic Broadway Station        NaN  34.052156   
81403S     81403S  Little Tokyo / Arts District Station        NaN  34.048845   

           stop_lon  stop_url  location_type parent_station tpis_name types  \
stop_id                                                                       
80101S  -118.192921       NaN              1            NaN       NaN   [0]   
80102S  -118.193700       NaN              1            NaN       NaN   [0]   
80105S  -118.189384       NaN              1            NaN       NaN   [0]   
80106S  -118.189382       NaN              1            NaN       NaN   [0]   
80107S  -118.189834       NaN              1            NaN       NaN   [0]   
...             ...       ...            ...            ...       ...   ...   
80708S  -118.335359       NaN              1            NaN       NaN   [0]   
80709S  -118.334851       NaN              1            NaN       NaN   [0]   
81401S  -118.251531       NaN              1            NaN       NaN   [0]   
81402S  -118.246321       NaN              1            NaN       NaN   [0]   
81403S  -118.238757       NaN              1            NaN       NaN   [0]   

             agency_id  
stop_id                 
80101S   [LACMTA_Rail]  
80102S   [LACMTA_Rail]  
80105S   [LACMTA_Rail]  
80106S   [LACMTA_Rail]  
80107S   [LACMTA_Rail]  
...                ...  
80708S   [LACMTA_Rail]  
80709S   [LACMTA_Rail]  
81401S   [LACMTA_Rail]  
81402S   [LACMTA_Rail]  
81403S   [LACMTA_Rail]  

[104 rows x 11 columns]

In [140]:
stop_to_types

{'640N': [1],
 'G15S': [1],
 '633N': [1],
 'D16N': [1],
 '134N': [1],
 '612S': [1],
 '635N': [1],
 '126N': [1],
 '621N': [1],
 '252N': [1],
 'R30N': [1],
 'R26S': [1],
 '640S': [1],
 '902S': [1],
 'A31S': [1],
 'M23S': [1],
 'A61S': [1],
 'M11S': [1],
 '255S': [1],
 '124S': [1],
 '626N': [1],
 '622S': [1],
 'A30N': [1],
 '135N': [1],
 'J20S': [1],
 '232S': [1],
 'H04N': [1],
 'D20N': [1],
 '256S': [1],
 'R34N': [1],
 'M09S': [1],
 'D08N': [1],
 '125N': [1],
 '111S': [1],
 'A22N': [1],
 'J30S': [1],
 'D21N': [1],
 'A17S': [1],
 'G16S': [1],
 'F21S': [1],
 'A12N': [1],
 'F25N': [1],
 '129N': [1],
 '412N': [1],
 'A33S': [1],
 'Q04N': [1],
 '234N': [1],
 'G20S': [1],
 'D03S': [1],
 '130N': [1],
 'F07S': [1],
 'B10S': [1],
 '723S': [1],
 'R21N': [1],
 'M18N': [1],
 'F06S': [1],
 '115S': [1],
 'D15S': [1],
 'D05N': [1],
 'A38S': [1],
 'A34N': [1],
 'F09N': [1],
 'M01S': [1],
 '716S': [1],
 'A19N': [1],
 'F14S': [1],
 '132S': [1],
 '713S': [1],
 '629N': [1],
 'R33N': [1],
 '614N': [1],
 'G19N

In [188]:
stops

stop_name   stop_lat   stop_lon  location_type  \
stop_id                                                                   
101      Van Cortlandt Park-242 St  40.889248 -73.898583            1.0   
103                         238 St  40.884667 -73.900870            1.0   
104                         231 St  40.878856 -73.904834            1.0   
106             Marble Hill-225 St  40.874561 -73.909831            1.0   
107                         215 St  40.869444 -73.915279            1.0   
...                            ...        ...        ...            ...   
S27                       Grasmere  40.603117 -74.084087            1.0   
S28                        Clifton  40.621319 -74.071402            1.0   
S29                      Stapleton  40.627915 -74.075162            1.0   
S30                  Tompkinsville  40.636949 -74.074835            1.0   
S31                      St George  40.643748 -74.073643            1.0   

        parent_station types  
stop_id                       
101                NaN   [1]  
103                NaN   [1]  
104                NaN   [1]  
106                NaN   [1]  
107                NaN   [1]  
...                ...   ...  
S27                NaN   [2]  
S28                NaN   [2]  
S29                NaN   [2]  
S30                NaN   [2]  
S31                NaN   [2]  

[499 rows x 6 columns]

In [106]:
stops

Empty DataFrame
Columns: []
Index: []

In [65]:
.join(, on="trip_id")

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat